<a href="https://colab.research.google.com/github/singhsa3/Emotions/blob/main/manualcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AvgPool2d
from torch.optim import Adam, SGD
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch


In [2]:
from google.colab import drive
drive.mount('/content/drive')

pathG='/content/drive/MyDrive/Pract/data' # @Reece, change this to your PathG
# @Reece, put greyimg.zip in your pathG

#pathG='../data'



#cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
#sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
with open(pathG+"/featuresdict.pickle", "rb") as f:
    features=pickle.load(f)
with open(pathG+"/filenameslist.pickle", "rb") as f:
    fnames=pickle.load(f)
import pandas as pd
df=pd.read_csv(pathG+"/labels/Yared Alemu_fear.csv")
df = df[(df['name'].isin (features.keys()))].reset_index()
df['name2'] =df['name'].apply (lambda x: x.split(".")[0]+".png")


In [4]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),    
            nn.AvgPool2d(kernel_size=2, stride=1), 

            nn.Conv2d(8, 16, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),                       
            AvgPool2d(kernel_size=3, stride=1),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            AvgPool2d(kernel_size=2, stride=1),         
            
        )
         
        self.linear_layers = Sequential(
            nn.Linear(1479200,25088),
            nn.ReLU(),
            nn.Linear(25088, 2048),
            nn.ReLU(),            
            nn.Linear(2048, 2048),
            nn.ReLU(),
            #nn.Linear(1479200,2048),
            #nn.ReLU(),
            nn.Linear(2048, 2),         
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [5]:
import zipfile
with zipfile.ZipFile(pathG+"/greyimg.zip", 'r') as zip_ref:
    zip_ref.extractall(pathG)

In [6]:
import glob
from PIL import Image
import os
imgPath= pathG+"/greyimg" 
filenames= glob.glob(imgPath+"/*.png" )
filenames = [os.path.basename(x) for x in filenames]

In [7]:
fl= imgPath+"/"+df.iloc[0]['name2']
x = (np.asarray(Image.open(fl))/255)
arr = np.expand_dims(x, axis=0)
#print(arr.shape)
for i,row in df.iterrows():
    if i>0:
        fl = imgPath+"/"+row['name2']
        ft = (np.asarray(Image.open(fl))/255)
        
        arr2 = np.expand_dims(ft, axis=0)
        #print(arr2.shape)
        arr = np.vstack((arr,arr2))
labels=np.array(df.emotion)

In [8]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(arr, labels, test_size = 0.3)

In [9]:
# converting training images into torch format
train_x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1], train_x.shape[2])
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)#.type(torch.FloatTensor)

# shape of training data
train_x.shape, train_y.shape
# converting validation images into torch format
val_x = val_x.reshape(val_x.shape[0], 1, val_x.shape[1], val_x.shape[2])
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([127, 1, 224, 224]), torch.Size([127]))

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.1)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
#print(model)
# Define the batch size and the number of epochs
BATCH_SIZE = 10
N_EPOCHS = 100
torch.cuda.empty_cache()
# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)
# empty list to store training losses
train_losses = [10000000,1000000,1000000,10000000,1000000]
# empty list to store validation losses
val_losses = []

In [ ]:
torch.cuda.empty_cache()

In [ ]:


# Loop over epochs
for epoch in range(N_EPOCHS):
    mnl = np.mean(train_losses)
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
          if torch.cuda.is_available():
            x_batch = x_batch.cuda()#.type(torch.cuda.FloatTensor)
            y_batch = y_batch.cuda()
            #x_val = val_x.cuda()
            #y_val = val_y.cuda()

            y_batch_pred = model(x_batch.float())
            #output_val = model(x_val)

            loss = criterion(y_batch_pred, y_batch)
            #loss_val = criterion(output_val, y_val)
            #train_losses.append(loss)
            #val_losses.append(loss_val)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Every 100 batches, print the loss for this batch
            # as well as the number of examples processed so far 
            if id_batch % 100 == 0:
                loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")    
    train_losses.pop()
    train_losses.insert(0,loss.detach().cpu().numpy())
    #if np.mean(train_losses)>mnl:
      #break
torch.cuda.empty_cache()



In [ ]:
from sklearn.metrics import accuracy_score
# prediction for validation set
predictions=[]
dataset = TensorDataset(val_x, val_y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        x_val = val_x.cuda()
        y_val = val_y.cuda()
        output = model(x_batch.float())
        softmax = torch.exp(output).cpu()
        prob = list(softmax.detach().cpu().numpy())
        prediction = np.argmax(prob, axis=1)
        predictions = predictions+list(prediction)

        

# accuracy on validation set
accuracy_score(val_y, np.array(predictions))